In [1]:
import os
import gc
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from scipy.stats import rankdata
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import textstat
from tqdm import tqdm
from typing import Dict, List, Tuple, NamedTuple, Callable
import scml
import mylib

In [2]:
class Conf(NamedTuple):
    device: torch.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    pretrained_dir: str = "pretrained/"
    dtfy_model_max_length: int = 512
    dtfy_batch_size: int = 256
    dtfy_models: Dict[str, str] = {
        "dto_": f"{pretrained_dir}unitaryai/detoxify/toxic_original-c1212f89.ckpt",
        "dtu_": f"{pretrained_dir}unitaryai/detoxify/toxic_debiased-c7548aa0.ckpt",
        "dtm_": f"{pretrained_dir}unitaryai/detoxify/multilingual_debiased-0b549669.ckpt"
    }
    dtfy_configs: Dict[str, str] = {
        "dto_": f"{pretrained_dir}bert-base-uncased",
        "dtu_": f"{pretrained_dir}roberta-base",
        "dtm_": f"{pretrained_dir}xlm-roberta-base"
    }
    tweeteval_model_max_length: int = 512
    tweeteval_batch_size: int = 64
    tweeteval_models: Dict[str, str] = {
        "te_roberta_off": f"{pretrained_dir}cardiffnlp/twitter-roberta-base-offensive",
        "te_roberta_emo_anger": f"{pretrained_dir}cardiffnlp/twitter-roberta-base-emotion",
        "te_roberta_snt_neg": f"{pretrained_dir}cardiffnlp/twitter-roberta-base-sentiment",
        "te_roberta_iro": f"{pretrained_dir}cardiffnlp/twitter-roberta-base-irony",
        "te_xlm_roberta_snt_neg": f"{pretrained_dir}cardiffnlp/twitter-xlm-roberta-base-sentiment",
    }
    tweeteval_label_index: Dict[str, int] = {
        "te_roberta_off": 1,
        "te_roberta_emo_anger": 0,
        "te_roberta_snt_neg": 0,
        "te_roberta_iro": 1,
        "te_xlm_roberta_snt_neg": 0,
    }
    hatebert_model_max_length: int = 512
    hatebert_batch_size: int = 128
    hatebert_models: Dict[str, str] = {
        "hb_bert_off": f"{pretrained_dir}hatebert/bert-offenseval",
        "hb_bert_abu" : f"{pretrained_dir}hatebert/bert-abuseval",
        "hb_hatebert_off": f"{pretrained_dir}hatebert/hatebert-offenseval",
        "hb_hatebert_abu" : f"{pretrained_dir}hatebert/hatebert-abuseval",
    }
    em_max_seq_length: int = 128
    em_batch_size: int = 1000
    em_models: Dict[str, str] = {
        "paraphrase-MiniLM-L6-v2": f"{pretrained_dir}sentence-transformers/paraphrase-MiniLM-L6-v2"
    }
        
        
conf = Conf()
print(conf)
if conf.device.type == 'cuda':
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')

Conf(device=device(type='cuda'), pretrained_dir='pretrained/', dtfy_model_max_length=512, dtfy_batch_size=256, dtfy_models={'dto_': 'pretrained/unitaryai/detoxify/toxic_original-c1212f89.ckpt', 'dtu_': 'pretrained/unitaryai/detoxify/toxic_debiased-c7548aa0.ckpt', 'dtm_': 'pretrained/unitaryai/detoxify/multilingual_debiased-0b549669.ckpt'}, dtfy_configs={'dto_': 'pretrained/bert-base-uncased', 'dtu_': 'pretrained/roberta-base', 'dtm_': 'pretrained/xlm-roberta-base'}, tweeteval_model_max_length=512, tweeteval_batch_size=64, tweeteval_models={'te_roberta_off': 'pretrained/cardiffnlp/twitter-roberta-base-offensive', 'te_roberta_emo_anger': 'pretrained/cardiffnlp/twitter-roberta-base-emotion', 'te_roberta_snt_neg': 'pretrained/cardiffnlp/twitter-roberta-base-sentiment', 'te_roberta_iro': 'pretrained/cardiffnlp/twitter-roberta-base-irony', 'te_xlm_roberta_snt_neg': 'pretrained/cardiffnlp/twitter-xlm-roberta-base-sentiment'}, tweeteval_label_index={'te_roberta_off': 1, 'te_roberta_emo_anger':

In [3]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()

In [4]:
df = pd.read_parquet("input/pre_ruddit.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5710 entries, 0 to 5709
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   5710 non-null   int32  
 1   bws     5710 non-null   float32
 2   worker  5710 non-null   int8   
 3   text    5710 non-null   object 
 4   text1   5710 non-null   object 
 5   text2   5710 non-null   object 
dtypes: float32(1), int32(1), int8(1), object(3)
memory usage: 184.1+ KB


# Character level features

In [5]:
%%time
col = "length"
df[col] = df["text1"].str.len()
df[col] = df[col].astype(np.int16)

Wall time: 5 ms


In [6]:
def digit_frac(row) -> float:
    return mylib.digit_frac(row["text1"])


def letter_frac(row) -> float:
    return mylib.letter_frac(row["text1"])


def space_frac(row) -> float:
    return mylib.space_frac(row["text1"])


def punc_frac(row) -> float:
    return mylib.punc_frac(row["text1"])


def upper_frac(row) -> float:
    return mylib.upper_frac(row["text1"])


char_fns: Dict[str, Callable] = {
    "digit_frac": digit_frac,
    "letter_frac": letter_frac,
    "space_frac": space_frac,
    "punc_frac": punc_frac,
    "upper_frac": upper_frac,
}

In [7]:
for col, fn in char_fns.items():
    print(col)
    df[col] = df.progress_apply(fn, axis=1)
    df[col] = df[col].astype(np.float32)

digit_frac


100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 36369.36it/s]


letter_frac


100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 35687.48it/s]


space_frac


100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 37808.12it/s]


punc_frac


100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 35241.79it/s]


upper_frac


100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 37565.51it/s]


# Textstat features

In [8]:
def syllable_count(row) -> int:
    return textstat.syllable_count(row["text1"])


def lexicon_count(row) -> int:
    return textstat.lexicon_count(row["text1"])


def sentence_count(row) -> int:
    return textstat.sentence_count(row["text1"])


def syllables_per_word(row) -> float:
    return row["syllable_count"] / (row["lexicon_count"] + 1)


def syllables_per_sent(row) -> float:
    return row["syllable_count"] / (row["sentence_count"] + 1)


def words_per_sent(row) -> float:
    return row["lexicon_count"] / (row["sentence_count"] + 1)


def flesch_reading_ease(row) -> float:
    return textstat.flesch_reading_ease(row["text1"])


def flesch_kincaid_grade(row) -> float:
    return textstat.flesch_kincaid_grade(row["text1"])


def gunning_fog(row) -> float:
    return textstat.gunning_fog(row["text1"])


def smog_index(row) -> float:
    return textstat.smog_index(row["text1"])


def automated_readability_index(row) -> float:
    return textstat.automated_readability_index(row["text1"])


def coleman_liau_index(row) -> float:
    return textstat.coleman_liau_index(row["text1"])


def linsear_write_formula(row) -> float:
    return textstat.linsear_write_formula(row["text1"])


def dale_chall_readability_score(row) -> float:
    return textstat.dale_chall_readability_score(row["text1"])


textstat_fns: Dict[str, Callable] = {
    "syllables_per_word": syllables_per_word,
    "syllables_per_sent": syllables_per_sent,
    "words_per_sent": words_per_sent,
    "flesch_reading_ease": flesch_reading_ease,
    "flesch_kincaid_grade": flesch_kincaid_grade,
    "gunning_fog": gunning_fog,
    "smog_index": smog_index,
    "automated_readability_index": automated_readability_index,
    "coleman_liau_index": coleman_liau_index,
    "linsear_write_formula": linsear_write_formula,
    "dale_chall_readability_score": dale_chall_readability_score,
}

In [9]:
col = "syllable_count"
df[col] = df.progress_apply(syllable_count, axis=1)
df[col] = df[col].astype(np.int32)

100%|█████████████████████████████████████████| 5710/5710 [00:00<00:00, 7573.43it/s]


In [10]:
col = "lexicon_count"
df[col] = df.progress_apply(lexicon_count, axis=1)
df[col] = df[col].astype(np.int32)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 65631.01it/s]


In [11]:
col = "sentence_count"
df[col] = df.progress_apply(sentence_count, axis=1)
df[col] = df[col].astype(np.int32)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 40778.11it/s]


In [12]:
for col, fn in textstat_fns.items():
    print(col)
    df[col] = df.progress_apply(fn, axis=1)
    df[col] = df[col].astype(np.float32)

syllables_per_word


100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 73184.71it/s]


syllables_per_sent


100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 86483.62it/s]


words_per_sent


...

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 24070.84it/s]


linsear_write_formula


100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 11992.74it/s]


dale_chall_readability_score


100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 10642.51it/s]


# TweetEval labels

In [13]:
sentences = list(df["text2"])
for col, model_dir in conf.tweeteval_models.items():
    tokenizer = AutoTokenizer.from_pretrained(
        model_dir, 
        model_max_length=conf.tweeteval_model_max_length
    )
    #print(f"{repr(tokenizer)}\nmodel_input_names={tokenizer.model_input_names}")
    x = tokenizer(sentences, truncation=True, padding="max_length")
    batches = torch.utils.data.DataLoader(mylib.Dataset(x), batch_size=conf.tweeteval_batch_size, shuffle=False)
    model = AutoModelForSequenceClassification.from_pretrained(model_dir)
    model.eval()
    model.to(conf.device)
    logits = None
    with torch.no_grad():
        for batch in tqdm(batches):
            for k, v in batch.items():
                batch[k] = v.to(conf.device)
            outputs = model(**batch)
            tmp = outputs.logits.detach().cpu()
            if logits is None:
                logits = tmp
            else:
                logits = torch.cat((logits, tmp), 0)
    logits = torch.nn.functional.softmax(logits, dim=1)
    print(f"{col} {logits.size()}\nlogits[:10]={logits[:10]}")
    df[col] = logits[:,conf.tweeteval_label_index[col]]
    df[col] = df[col].astype(np.float32)
    del tokenizer, model
    gc.collect()

100%|███████████████████████████████████████████████| 90/90 [04:10<00:00,  2.79s/it]


te_roberta_off torch.Size([5710, 2])
logits[:10]=tensor([[0.9025, 0.0975],
        [0.5804, 0.4196],
        [0.6679, 0.3321],
        [0.8045, 0.1955],
        [0.7823, 0.2177],
        [0.7749, 0.2251],
        [0.8272, 0.1728],
        [0.6837, 0.3163],
        [0.7598, 0.2402],
        [0.9035, 0.0965]])


100%|███████████████████████████████████████████████| 90/90 [04:06<00:00,  2.74s/it]


te_roberta_emo_anger torch.Size([5710, 4])
logits[:10]=tensor([[0.0376, 0.0565, 0.8628, 0.0431],
        [0.1369, 0.1012, 0.2558, 0.5061],
        [0.8166, 0.0105, 0.0391, 0.1338],
        [0.9419, 0.0041, 0.0231, 0.0308],
        [0.6140, 0.0225, 0.1561, 0.2074],
        [0.8744, 0.0085, 0.0585, 0.0586],
        [0.8556, 0.0071, 0.0948, 0.0425],
        [0.8475, 0.0155, 0.0260, 0.1111],
        [0.8054, 0.0086, 0.0704, 0.1156],
        [0.1417, 0.3192, 0.3703, 0.1687]])


100%|███████████████████████████████████████████████| 90/90 [04:05<00:00,  2.73s/it]


te_roberta_snt_neg torch.Size([5710, 3])
logits[:10]=tensor([[0.0407, 0.7561, 0.2032],
        [0.4268, 0.5488, 0.0244],
        [0.7662, 0.2259, 0.0080],
        [0.6066, 0.3729, 0.0206],
        [0.5169, 0.4232, 0.0599],
        [0.5795, 0.3848, 0.0357],
        [0.3226, 0.6394, 0.0380],
        [0.7787, 0.2147, 0.0066],
        [0.7998, 0.1863, 0.0139],
        [0.1055, 0.6091, 0.2854]])


100%|███████████████████████████████████████████████| 90/90 [04:05<00:00,  2.73s/it]


te_roberta_iro torch.Size([5710, 2])
logits[:10]=tensor([[0.7999, 0.2001],
        [0.4219, 0.5781],
        [0.3392, 0.6608],
        [0.8240, 0.1760],
        [0.8869, 0.1131],
        [0.4666, 0.5334],
        [0.7453, 0.2547],
        [0.0874, 0.9126],
        [0.8681, 0.1319],
        [0.6555, 0.3445]])


100%|███████████████████████████████████████████████| 90/90 [04:05<00:00,  2.73s/it]


te_xlm_roberta_snt_neg torch.Size([5710, 3])
logits[:10]=tensor([[0.1049, 0.7652, 0.1299],
        [0.1865, 0.7562, 0.0573],
        [0.4178, 0.4738, 0.1084],
        [0.7074, 0.2630, 0.0296],
        [0.8242, 0.1541, 0.0217],
        [0.5170, 0.4337, 0.0493],
        [0.5321, 0.4271, 0.0408],
        [0.5248, 0.4357, 0.0396],
        [0.9013, 0.0858, 0.0129],
        [0.3988, 0.4830, 0.1182]])


# HateBert labels

In [14]:
# all Hatebert models use the same tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    conf.hatebert_models["hb_hatebert_off"], 
    model_max_length=conf.hatebert_model_max_length
)
print(f"{repr(tokenizer)}\nmodel_input_names={tokenizer.model_input_names}")

PreTrainedTokenizerFast(name_or_path='pretrained/hatebert/hatebert-offenseval', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
model_input_names=['input_ids', 'token_type_ids', 'attention_mask']


In [15]:
%%time
x = tokenizer(sentences, truncation=True, padding="max_length")
print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
len=5710
Wall time: 972 ms


In [16]:
batches = torch.utils.data.DataLoader(mylib.Dataset(x), batch_size=conf.hatebert_batch_size, shuffle=False)
for col, model_dir in conf.hatebert_models.items():    
    model = AutoModelForSequenceClassification.from_pretrained(model_dir)
    model.eval()
    model.to(conf.device)
    logits = None
    with torch.no_grad():
        for batch in tqdm(batches):
            for k, v in batch.items():
                batch[k] = v.to(conf.device)
            outputs = model(**batch)
            tmp = outputs.logits.detach().cpu()
            if logits is None:
                logits = tmp
            else:
                logits = torch.cat((logits, tmp), 0)
    logits = torch.nn.functional.softmax(logits, dim=1)
    print(f"{col} {logits.size()}\nlogits[:10]={logits[:10]}")
    df[col] = logits[:,1]
    df[col] = df[col].astype(np.float32)

100%|███████████████████████████████████████████████| 45/45 [04:10<00:00,  5.56s/it]


hb_bert_off torch.Size([5710, 2])
logits[:10]=tensor([[0.9598, 0.0402],
        [0.7718, 0.2282],
        [0.7567, 0.2433],
        [0.9064, 0.0936],
        [0.8356, 0.1644],
        [0.7731, 0.2269],
        [0.5952, 0.4048],
        [0.8502, 0.1498],
        [0.7770, 0.2230],
        [0.9258, 0.0742]])


100%|███████████████████████████████████████████████| 45/45 [04:11<00:00,  5.59s/it]


hb_bert_abu torch.Size([5710, 2])
logits[:10]=tensor([[0.9910, 0.0090],
        [0.9717, 0.0283],
        [0.9545, 0.0455],
        [0.9631, 0.0369],
        [0.9727, 0.0273],
        [0.8744, 0.1256],
        [0.9001, 0.0999],
        [0.8544, 0.1456],
        [0.8708, 0.1292],
        [0.9881, 0.0119]])


100%|███████████████████████████████████████████████| 45/45 [04:13<00:00,  5.62s/it]


hb_hatebert_off torch.Size([5710, 2])
logits[:10]=tensor([[0.9377, 0.0623],
        [0.5526, 0.4474],
        [0.7553, 0.2447],
        [0.7828, 0.2172],
        [0.8740, 0.1260],
        [0.9279, 0.0721],
        [0.8304, 0.1696],
        [0.7886, 0.2114],
        [0.8038, 0.1962],
        [0.8965, 0.1035]])


100%|███████████████████████████████████████████████| 45/45 [04:12<00:00,  5.60s/it]

hb_hatebert_abu torch.Size([5710, 2])
logits[:10]=tensor([[0.9846, 0.0154],
        [0.9717, 0.0283],
        [0.9570, 0.0430],
        [0.9699, 0.0301],
        [0.9700, 0.0300],
        [0.9775, 0.0225],
        [0.9800, 0.0200],
        [0.9733, 0.0267],
        [0.9407, 0.0593],
        [0.9795, 0.0205]])


# Detoxify labels

In [17]:
gc.collect()
dtfy_fs = []
for prefix, checkpoint in tqdm(conf.dtfy_models.items()):
    res = mylib.detoxify_labels(
        sentences,
        checkpoint=checkpoint,
        config_dir=conf.dtfy_configs[prefix],
        model_max_length=conf.dtfy_model_max_length,
        device=conf.device,
        batch_size=conf.dtfy_batch_size
    )
    for k, v in res.items():
        col = prefix + k
        df[col] = v
        df[col] = df[col].astype(np.float32)
        dtfy_fs.append(col)
    gc.collect()

100%|█████████████████████████████████████████████████| 3/3 [04:04<00:00, 81.65s/it]


In [18]:
print(dtfy_fs)

['dto_toxicity', 'dto_severe_toxicity', 'dto_obscene', 'dto_threat', 'dto_insult', 'dto_identity_attack', 'dtu_toxicity', 'dtu_severe_toxicity', 'dtu_obscene', 'dtu_identity_attack', 'dtu_insult', 'dtu_threat', 'dtu_sexual_explicit', 'dtm_toxicity', 'dtm_severe_toxicity', 'dtm_obscene', 'dtm_identity_attack', 'dtm_insult', 'dtm_threat', 'dtm_sexual_explicit']


# Embeddings

In [19]:
model = SentenceTransformer(conf.em_models["paraphrase-MiniLM-L6-v2"], device=conf.device)
model.max_seq_length = conf.em_max_seq_length
em = model.encode(sentences=sentences, batch_size=conf.em_batch_size, show_progress_bar=True, convert_to_numpy=True)
print(f"em.shape={em.shape}")

[INFO|SentenceTransformer.py:60] 2022-01-19 08:12:48,212 >> Load pretrained SentenceTransformer: pretrained/sentence-transformers/paraphrase-MiniLM-L6-v2
[INFO|SentenceTransformer.py:60] 2022-01-19 08:12:48,212 >> Load pretrained SentenceTransformer: pretrained/sentence-transformers/paraphrase-MiniLM-L6-v2


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

em.shape=(5710, 384)


In [20]:
%%time
em_size = em.shape[1]
em_cols = [f"zz{i:04d}" for i in range(em_size)]
df[em_cols] = em
df[em_cols] = df[em_cols].astype(np.float32)
del sentences

s:\dev\seahrh\kaggle-jigsaw-toxic-severity-rating\env\lib\site-packages\pandas\core\frame.py:3673: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


Wall time: 258 ms


# Review data

In [21]:
cols = ["label", "bws", "worker", "length"]
cols += list(char_fns.keys())
cols += list(textstat_fns.keys())
cols += dtfy_fs
cols += list(conf.hatebert_models.keys()) 
cols += list(conf.tweeteval_models.keys()) 
df[cols].describe(percentiles=percentiles)

,label,bws,worker,length,digit_frac,letter_frac,space_frac,punc_frac,upper_frac,syllables_per_word,syllables_per_sent,words_per_sent,flesch_reading_ease,flesch_kincaid_grade,gunning_fog,smog_index,automated_readability_index,coleman_liau_index,linsear_write_formula,dale_chall_readability_score,dto_toxicity,dto_severe_toxicity,dto_obscene,dto_threat,dto_insult,dto_identity_attack,dtu_toxicity,dtu_severe_toxicity,dtu_obscene,dtu_identity_attack,dtu_insult,dtu_threat,dtu_sexual_explicit,dtm_toxicity,dtm_severe_toxicity,dtm_obscene,dtm_identity_attack,dtm_insult,dtm_threat,dtm_sexual_explicit,hb_bert_off,hb_bert_abu,hb_hatebert_off,hb_hatebert_abu,te_roberta_off,te_roberta_emo_anger,te_roberta_snt_neg,te_roberta_iro,te_xlm_roberta_snt_neg
count,5710.00000,5710.000000,5710.0,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5.710000e+03,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000
mean,2855.50000,-0.027706,0.0,196.434501,0.003538,0.787127,0.173322,0.036013,0.031820,1.306213,13.194283,9.516296,75.075684,6.465289,8.678009,3.174711,7.575517,7.008681,7.984055,8.397984,0.177655,0.012897,0.116283,0.008192,0.058391,0.011024,0.198440,4.811980e-03,0.116563,0.015244,0.082472,0.010910,0.041993,0.205532,0.006158,0.109385,0.013857,0.083933,0.014859,0.047163,0.335537,0.167246,0.328364,0.142462,0.347343,0.504049,0.505597,0.305937,0.579142
std,1648.47935,0.334195,0.0,171.036190,0.012874,0.034742,0.020724,0.025652,0.048489,0.199887,7.814633,5.374849,19.881145,4.232607,4.458600,4.696062,5.256020,3.881966,5.502464,2.272001,0.327492,0.051897,0.285911,0.055522,0.171945,0.062557,0.345171,2.272858e-02,0.290018,0.074101,0.209081,0.069112,0.154768,0.348464,0.029047,0.278292,0.076570,0.215375,0.078996,0.161885,0.334126,0.279449,0.284541,0.219889,0.251188,0.358066,0.318980,0.272433,0.300423
min,1.00000,-0.889000,0.0,14.000000,0.000000,0.400000,0.040541,0.000000,0.000000,0.800000,2.000000,1.500000,-48.980000,-2.500000,1.200000,0.000000,-6.800000,-10.160000,0.500000,0.200000,0.000506,0.000080,0.000141,0.000086,0.000164,0.000121,0.000286,9.529070e-07,0.000017,0.000052,0.000061,0.000012,0.000009,0.000186,0.000009,0.000063,0.000045,0.000095,0.000014,0.000012,0.008860,0.002739,0.006939,0.007050,0.022149,0.004901,0.000799,0.014366,0.007853
1%,58.09000,-0.667000,0.0,24.000000,0.000000,0.666667,0.111300,0.000000,0.000000,0.857143,2.500000,2.333333,19.439399,-1.500000,1.600000,0.000000,-2.800000,-2.680100,1.250000,0.350000,0.000554,0.000087,0.000158,0.000096,0.000171,0.000134,0.000359,1.082210e-06,0.000021,0.000063,0.000091,0.000015,0.000011,0.000280,0.000013,0.000094,0.000065,0.000149,0.000021,0.000017,0.013538,0.003761,0.021530,0.010232,0.049472,0.010917,0.001821,0.022453,0.019090
5%,286.45000,-0.521000,0.0,33.000000,0.000000,0.729363,0.137931,0.008264,0.007042,1.000000,4.000000,3.000000,41.606500,0.500000,2.400000,0.000000,0.345000,0.760000,2.000000,5.689000,0.000612,0.000093,0.000165,0.000104,0.000175,0.000138,0.000409,1.200621e-06,0.000024,0.000070,0.000102,0.000017,0.000012,0.000352,0.000016,0.000115,0.000078,0.000177,0.000027,0.000020,0.021280,0.005180,0.036103,0.013510,0.073817,0.023043,0.008893,0.035101,0.054203
10%,571.90000,-0.426000,0.0,42.000000,0.000000,0.747601,0.148148,0.013889,0.009479,1.058824,4.666667,3.500000,50.680001,1.700000,3.200000,0.000000,1.700000,2.200000,2.500000,6.258000,0.000663,0.000097,0.000170,0.000108,0.000177,0.000140,0.000452,1.270154e-06,0.000026,0.000076,0.000111,0.000019,0.000013,0.000410,0.000019,0.000129,0.000086,0.000199,0.000030,0.000022,0.028715,0.006399,0.049363,0.015942,0.091455,0.0388

In [22]:
cols += em_cols
df[cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5710 entries, 0 to 5709
Data columns (total 433 columns):
 #    Column                        Non-Null Count  Dtype  
---   ------                        --------------  -----  
 0    label                         5710 non-null   int32  
 1    bws                           5710 non-null   float32
 2    worker                        5710 non-null   int8   
 3    length                        5710 non-null   int16  
 4    digit_frac                    5710 non-null   float32
 5    letter_frac                   5710 non-null   float32
 6    space_frac                    5710 non-null   float32
 7    punc_frac                     5710 non-null   float32
 8    upper_frac                    5710 non-null   float32
 9    syllables_per_word            5710 non-null   float32
 10   syllables_per_sent            5710 non-null   float32
 11   words_per_sent                5710 non-null   float32
 12   flesch_reading_ease           5710 non-null   

In [23]:
%%time
df[cols].to_parquet("output/tra.parquet", index=False)

Wall time: 212 ms
